Use this utlity to update the returns and std_dev fields within investment-options.csv

In [22]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [23]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [24]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [25]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [26]:
# read in sp500.csv
sp500 = pd.read_csv('sp500.csv')
sp500.drop(columns=['SEC filings', 'Headquarters Location', 'Date first added','CIK', 'Founded'], inplace=True)
sp500.rename(columns={'Security':'Description',
                      'GICS Sector':'Asset Class'}, inplace=True)
sp500.set_index("Symbol", inplace=True)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M Company,Industrials,Industrial Conglomerates
ABT,Abbott Laboratories,Health Care,Health Care Equipment
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals
ABMD,ABIOMED Inc,Health Care,Health Care Equipment
ACN,Accenture plc,Information Technology,IT Consulting & Other Services
...,...,...,...
YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,Health Care,Health Care Equipment


In [27]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [28]:
# map sp500 GICS sectors to brownbear defined asset classes
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp500['Asset Class'] = sp500.apply(_asset_class, axis=1)

# yahoo finance uses '-' where '.' is used in symbol names
sp500.index = sp500.index.str.replace('.', '-')
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M Company,US Stocks:Industrials,Industrial Conglomerates
ABT,Abbott Laboratories,US Stocks:Healthcare,Health Care Equipment
ABBV,AbbVie Inc.,US Stocks:Healthcare,Pharmaceuticals
ABMD,ABIOMED Inc,US Stocks:Healthcare,Health Care Equipment
ACN,Accenture plc,US Stocks:Technology,IT Consulting & Other Services
...,...,...,...
YUM,Yum! Brands Inc,US Stocks:Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,US Stocks:Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,US Stocks:Healthcare,Health Care Equipment


In [29]:
# make symbols list
symbols = list(sp500.index)
#symbols

In [30]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

.........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................


In [31]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df.tail()[['NVDA', 'WST', 'ADBE', 'NOW', 'AMD', 'PYPL', 'AAPL', 'MSFT', 'DHR', 'TMO', 'AMZN', 'DXCM', 'ODFL', 'REGN', 'ROL', 'FDX', 'TMUS', 'CDNS', 'CRM', 'MSCI']]

,NVDA,WST,ADBE,NOW,AMD,PYPL,AAPL,MSFT,DHR,TMO,AMZN,DXCM,ODFL,REGN,ROL,FDX,TMUS,CDNS,CRM,MSCI
Date,,,,,,,,,,,,,,,,,,,,
2021-02-10,590.57,292.29,492.67,589.36,92.35,283.18,135.39,242.26,237.49,484.88,3286.58,399.44,208.99,491.22,37.93,257.46,124.49,137.86,236.72,418.91
2021-02-11,610.03,291.09,496.62,594.47,92.66,285.03,135.13,243.93,240.66,494.05,3262.13,410.20,208.04,490.54,37.65,254.36,124.40,143.97,241.24,426.10
2021-02-12,598.45,294.01,498.84,591.29,93.77,298.37,135.37,244.43,245.90,507.83,3277.71,412.56,212.21,485.62,37.59,263.00,123.68,145.68,240.37,434.00
2021-02-16,613.21,295.61,501.64,585.74,91.46,304.79,133.19,243.14,242.27,500.08,3268.95,404.38,210.19,480.71,36.84,261.83,122.00,145.99,248.59,440.14
2021-02-17,596.24,293.99,491.23,580.94,89.94,297.20,130.84,244.20,239.91,498.46,3308.64,414.12,207.99,481.68,36.60,259.09,121.30,141.78,247.66,438.54


In [32]:
# sample symbol
symbol = 'MMM'

In [33]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

15.231359352590568

In [34]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [35]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.3099159082446824

In [36]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.15979394399474325

In [37]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   135.60
2015-02-28   141.78
2015-03-31   138.67
2015-04-30   131.48
2015-05-31   134.58
2015-06-30   130.54
2015-07-31   128.04
2015-08-31   121.09
2015-09-30   120.77
2015-10-31   133.93
2015-11-30   134.27
2015-12-31   129.17
2016-01-31   129.48
2016-02-29   135.49
2016-03-31   143.92
2016-04-30   144.56
2016-05-31   146.34
2016-06-30   152.26
2016-07-31   155.07
2016-08-31   156.81
2016-09-30   154.18
2016-10-31   144.61
2016-11-30   151.21
2016-12-31   157.23
2017-01-31   153.92
2017-02-28   165.14
2017-03-31   169.56
2017-04-30   173.55
2017-05-31   182.28
2017-06-30   185.60
2017-07-31   179.34
2017-08-31   183.20
2017-09-30   188.20
2017-10-31   206.39
2017-11-30   219.11
2017-12-31   212.10
2018-01-31   225.74
2018-02-28   213.48
2018-03-31   198.99
2018-04-30   176.21
2018-05-31   180.00
2018-06-30   179.54
2018-07-31   193.78
2018-08-31   193.79
2018-09-30   193.60
2018-10-31   174.81
2018-11-30   192.34
2018-12-31   176.26
2019-01-31   185.29
2019-02-28   19

In [38]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     nan
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2020-10-31   -0.00
2020-11-30    0.09
2020-12-31    0.01
2021-01-31    0.00
2021-02-28    0.01
Freq: M, Name: MMM, Length: 74, dtype: float64

In [39]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.22585567958265657

In [40]:
# read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: S&P 500 investment options. 09/29/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [41]:
# for each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = lines.copy()

# This is still slow (2.53 s)
for i, (index, row) in enumerate(sp500.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [42]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')